In [33]:
import cv2
import numpy as np
from ultralytics import YOLO

In [37]:
model = YOLO("yolov8s.pt")  # Ensure "yolov8s.pt" is the correct path to the model file.
video_path = "traffic.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("❌ Error: Could not open video file.")
    exit()

In [38]:
line_y = 300  
vehicle_count = 0
vehicle_ids = set()  # To track unique vehicles
vehicle_types = {"car": 0, "truck": 0, "bus": 0, "motorcycle": 0}
LINE_COLOR = (0, 255, 0)  
TEXT_COLOR = (0, 255, 255)  
BOX_COLOR = (255, 255, 255)  # Blue for bounding box

In [39]:
vehicle_classes = {2: "car", 3: "motorcycle", 5: "bus", 7: "truck"}
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("✅ End of video or error reading frame.")
        break

    # Perform object detection using YOLOv8
    results = model.predict(frame, conf=0.3)  # Lower confidence threshold

    for r in results:
        for box in r.boxes.cpu().numpy():  # Get bounding boxes
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Extract coordinates
            conf = box.conf[0]  # Confidence score
            class_id = int(box.cls[0])  # Object class (e.g., car, truck, motorcycle)

            # Debug: Log all detected objects
            print(f"Detected {vehicle_classes.get(class_id, 'unknown')} | Class ID: {class_id}, Confidence: {conf:.2f}")

            # Check if detected object is a vehicle
            if class_id in vehicle_classes.keys():
                vehicle_type = vehicle_classes[class_id]
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2

                # Debugging for Trucks and Others
                if vehicle_type == "truck":
                    print(f"Truck Detected at ({center_x}, {center_y}) with Confidence: {conf:.2f}")

                # Refined counting logic for vehicles crossing the line
                if (line_y - 20 < center_y < line_y + 20) and (center_x, vehicle_type) not in vehicle_ids:
                    vehicle_ids.add((center_x, vehicle_type))  # Include vehicle type in the ID
                    vehicle_count += 1
                    vehicle_types[vehicle_type] += 1
                    print(f"Counted {vehicle_type} at ({center_x}, {center_y})")

                # Draw bounding box around the detected vehicle
                cv2.rectangle(frame, (x1, y1), (x2, y2), BOX_COLOR, 2)

                # Label the bounding box with class name and confidence
                label = f"{vehicle_type.capitalize()} ({conf:.2f})"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, BOX_COLOR, 2)
                # Draw counting line on the frame
    cv2.line(frame, (0, line_y), (frame.shape[1], line_y), LINE_COLOR, 2)

    # Display vehicle counts on a black overlay
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (frame.shape[1], 100), (0, 0, 0), -1)  # Black rectangle
    alpha = 0.7  # Transparency factor
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # Display the total vehicle count and individual type counts
    cv2.putText(frame, f"Total Vehicles: {vehicle_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, TEXT_COLOR, 2)
    y_offset = 70
    for vehicle_type, count in vehicle_types.items():
        cv2.putText(frame, f"{vehicle_type.capitalize()}: {count}", (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, TEXT_COLOR, 2)
        y_offset += 30
# Show the output frame
    cv2.imshow("Vehicle Detection & Counting", frame)

    # Exit the program when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 7 persons, 25 cars, 4 motorcycles, 4 buss, 1 truck, 173.9ms
Speed: 1.6ms preprocess, 173.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Detected bus | Class ID: 5, Confidence: 0.91
Detected car | Class ID: 2, Confidence: 0.89
Detected car | Class ID: 2, Confidence: 0.88
Detected car | Class ID: 2, Confidence: 0.87
Detected car | Class ID: 2, Confidence: 0.85
Detected car | Class ID: 2, Confidence: 0.84
Detected truck | Class ID: 7, Confidence: 0.84
Truck Detected at (925, 545) with Confidence: 0.84
Detected car | Class ID: 2, Confidence: 0.82
Detected car | Class ID: 2, Confidence: 0.79
Detected car | Class ID: 2, Confidence: 0.77
Detected car | Class ID: 2, Confidence: 0.75
Detected unknown | Class ID: 0, Confidence: 0.72
Detected car | Class ID: 2, Confidence: 0.72
Detected car | Class ID: 2, Confidence: 0.70
Detected car | Class ID: 2, Confidence: 0.70
Detected unknown | Class ID: 0, Confidence: 0.69
Detected bus | Class ID: 5, Confidence: 0.68
Dete

In [40]:

print("Final Vehicle Count Summary:")
print(f"Total Vehicles: {vehicle_count}")
for vehicle_type, count in vehicle_types.items():
    print(f"{vehicle_type.capitalize()}: {count}")

Final Vehicle Count Summary:
Total Vehicles: 115
Car: 96
Truck: 1
Bus: 7
Motorcycle: 11
